In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
%config Completer.use_jedi = False
import tqdm
%config InlineBackend.figure_format = 'retina'
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
#mpl.rcParams['figure.figsize'] = (6, 4)
import numpy as np
import h5py

%load_ext autoreload
%autoreload 2

In [45]:
with h5py.File(f"./primegaps_sampling_5_100.hdf5", "r") as f:
    box = {k:v[:] for k,v in f.items()}

M = 1001
N = box['counts'].shape[0]
data = {str(k+5):box['counts'][k][:M] for k in range(box['counts'].shape[0])}
data['gaps'] = box['gaps'][:M]
data['existing'] = data['5']

#https://stackoverflow.com/questions/15347174/python-finding-prime-factors
def number_unique_prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            if i not in factors:
                factors.append(i)
    if n > 1:
        if n not in factors:
            factors.append(n)
    return len(factors)

#colors = [plt.cm.virid]
data['nupf'] = np.array([str(number_unique_prime_factors(i)) for i in data['gaps']])

In [47]:
#column_datasource.py
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Slider, CustomJS, CategoricalColorMapper
from bokeh.models.tools import HoverTool
from bokeh.layouts import column
from bokeh.palettes import d3

palette = d3['Category10'][len(np.unique(data['nupf']))]
color_map = CategoricalColorMapper(factors=np.unique(data['nupf']),palette=palette)

output_file('primegaps.html')
source = ColumnDataSource(data=data)

# Add the tooltips when you hover over a patch
TOOLTIPS = [
    ("Number", "@gaps"),
    ("# of unique prime factors", "@nupf")
]

p = figure(title='The distribution of gaps between the first 1,000 primes after 10^Magnitude',plot_width=1000, plot_height=400, y_axis_type="log",
           background_fill_color="#fafafa",tooltips=TOOLTIPS)
p.y_range.start = 0.3
p.y_range.end = 3000
p.x_range.start = 0
p.x_range.end = 1000
p.xaxis.axis_label = 'Size of gap'
p.yaxis.axis_label = 'Frequency'

# add a circle renderer with a size, color, and alpha
p.circle('gaps', 'existing', size=4,
          color={'field': 'nupf', 'transform': color_map}, alpha=1.0, source=source)


# Add a slider to control which year the patches are coloured by
magnitude_slider = Slider(start=5, end=N+5, value=5, step=1, title="Magnitude")

# Add the custom javascript to interface the slider with the patches
callback = CustomJS(args=dict(source=source, magnitude=magnitude_slider), code="""
        var data = source.data;
        var new_magnitude = magnitude.value;
        var existing = data['existing']
        var replacement = data[String(new_magnitude)]
        for (var index = 0; index < existing.length; index++) {
            existing[index] = replacement[index]
        }
        source.change.emit();
    """)
magnitude_slider.js_on_change('value', callback)

# Combine plot and slider and output
layout = column(p,magnitude_slider)
show(layout)
